# AI - Final - Computer Assignment
## Hossein Entezari Zarch - 810196419
## Summer - 2020


In [74]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import mean_absolute_error as MAE
from __future__ import unicode_literals
import hazm
from torch.utils.data import Dataset, DataLoader, random_split, Subset
from torch.utils.data.sampler import SubsetRandomSampler
from torch.optim import Adam
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
from datetime import timedelta
from datetime import datetime
from tqdm import trange, tqdm
from matplotlib import pyplot as plt

 # Introduction
<p dir='rtl'>
    مقدمه: هدف ما در اینجا به طور کلی ساختن یک تابع(مدل پیش‌بینی) است که بتواند با استفاده از داده‌هایی که به عنوان دیتاست به ما داده شده است، به خوبی الگوهای موجود در داده‌ها را کشف کند و آنها را یاد بگیرد. این مدل توانایی خواهد داشت که با دیدن و یادگیری روی داده‌هایی که ما به آن می‌دهیم مثال‌هایی که تا به‌حال ندیده را پیش‌بینی و حل کند.
    <br><br>
    روند کلی: روند کلی ای که ما برای انجام این فرآیند پیش می‌گیریم شامل: ۱-پیش‌پردازش داده‌ها و استخراج ویژگی‌ها ۲-جدا کردن بخشی از داده ها به عنوان داده آزمایش ۳-تعریف مدل و انجام عملیات یادگیری روی آن ۴-پیش‌بینی داده‌های آزمایش و مقایسه با مقادیر واقعی هدف
    <br><br>
</p>

# PreProcess & Feature Extraction
<p dir='rtl'>
    به طور کلی در اینجا هدف ما تبدیل داده‌ها به شکلی است که برای مدل ما قابل فهم‌تر باشند و بیشتر مدل بتواند از آنها اطلاعات یاد بگیرد، به منظور این هدف ما فیلد‌های مختلف داده‌ها را به صورت های مختلف تحلیل کرده و ذخیره می‌کنیم و همچنین در این مرحله داده‌هایی مقادیر نامناسب(داده پرت) هستند را حذف می‌کنیم.
    <br><br>
    encodeing متغیر های گسسته:
    به طور کلی در ساده ترین راه‌ها برای encode
    کردن این نوع داده‌ها، دو روش Labeling, OneHotEncoding
    استفاده م‌شود که به طور خلاصه در روش Labeling 
    هر مقدار از متغیر به یک عدد یکتا مپ می‌شود و آن عدد به جای آن مقدار ذخیره می‌شود و در روش OneHotEncoding
    به ازای هر مقدار خاص از متغیر یک جایگاه اختصاص داده می‌شود که اگر آن مقدار رخ داده جایگاه متناظرش ۱ می‌شود و بقیه جایگاه‌ها صفر می‌مانند.
    <br><br>
    ما در اینجا در encode 
    کردن {برند، شهر} از روش OneHotEncoding 
    استفاده کردیم زیرا با توجه به این که از مدل MultiLayerPerceptronRegressor
    استفاده می‌کنیم و خروجی در این مدل در اصل ترکیبی غیرخطی(وجود تابع غیرخطی بین لایه‌ها) از مقادیر ورودی است لذا مقادیر مختلف معنای فاصله بین‌شان تعریف نمی‌شود و اگر دو مقدار Index
    متناظرشان نزدیک هم باشد یا یکی از دیگری بزرگ‌تر باشد معنایی ندارد ولی این می‌تواند شبکه را به اشتباه بیندازد ولی در روش استفاده شده ما همه مقادیر ارزش و اعتبار یکسان دارند و این تنها وزن‌های شبکه است که میان آنها تمایز ایجاد می‌کند. پش با این استدلالات فیلدهای {برند، شهر} که تعداد مقادیرشان نیز زیاد نیست با OneHotEncoding
    انکود کردیم.
</p>

In [75]:
def one_hot_encode_brand(data):
    enc = OneHotEncoder(handle_unknown='ignore')
    enc_df = pd.DataFrame(enc.fit_transform(data[['brand']]).toarray())
    enc_df.columns = enc.get_feature_names(['brand'])
    data = data.join(enc_df)
    data = data.drop('brand', axis=1)
    return data

def one_hot_encode_city(data):
    enc = OneHotEncoder(handle_unknown='ignore')
    enc_df = pd.DataFrame(enc.fit_transform(data[['city']]).toarray())
    enc_df.columns = enc.get_feature_names(['city'])
    data = data.join(enc_df)
    data = data.drop('city', axis=1)
    return data

# Cycle Encode
<p dir='rtl'>
  در مورد فیلد {روز و ساعت}، به طور کلی ما از اهمیت این داده‌ها در فرآیند یادگیری و میزان تاثیر آنها در یادگیری مدل (میزان وابستگی قیمت به این فیلد) آگاه نیستیم ولی برای اطمینان و با توجه به این که مدل ما در صورت نیاز از این داده استفاده خواهد کرد و یا در غیر این صورت خیر، پس ابتدا باید داده‌ها را به صورتی که برای مدل به راحتی قابل فهم باشد در می‌آوریم  سپس آنها را در داده ذخیره کنیم.
    <br><br>
    Cyclic Encoding:
    می‌دانیم که داده‌های مربوط به تاریخ و ساعت هم توالی دارند و هم تکرار یعنی به عنوان مثال شنبه بسیار به جمعه نزدیک است و این توالی ادامه دارد لذا بهترین راهی که به نظر من رسید، این بود که این داده‌ها را در تابع sin, cos
    که توابعی متناوب هستند مدل کنیم. برای این مورد هر کدام از ساعت و روز را جداگانه با فاصله مساوی زاویه‌ای روی دایره مثلثاتی تعریف کردیم و sin, cos
    متناظر آن زاویه را در نظر گرفتیم.
    <br><br>
    پیاده‌سازی: برای پیاده‌سازی این هدف، ابتدا لیست مقادیر متمایز این متغیر ها را تهیه کردیم و سپس هر کدام از آنها را به ترتیب و با فاصله یکسان روی دایره مثلثاتی قرار داده و لیست سینوس و کیسنوس متناظرشان را ساختیم و در نهایت با مراجه به آن لیست، مقادیر این فیلد را تبدیل و جایگزین فیلد اولیه کردیم.
</p>

In [76]:
def cycle_encode(values, value):
    return [np.sin(2*np.pi*values.index(value)/len(values)), np.cos(2*np.pi*values.index(value)/len(values))]

def make_week_day_cycle_list():
    ans = {}
    values = ['Saturday', 'Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
    for value in values:
        ans[value] = cycle_encode(values, value)
    return ans

def make_hour_cycle_list():
    ans = {}
    values = ['01AM', '02AM', '03AM', '04AM', '05AM', '06AM', '07AM', '08AM', '09AM', '10AM', '11AM', '12AM',
              '01PM', '02PM', '03PM', '04PM', '05PM', '06PM', '07PM', '08PM', '09PM', '10PM', '11PM', '12PM']
    for value in values:
        ans[value] = cycle_encode(values, value)
    return ans

def created_at_encode(data):
    day_sin = []
    day_cos = []
    sinc = make_week_day_cycle_list()
    
    hour_sin = []
    hour_cos = []
    hours = make_hour_cycle_list()
    
    for element in data['created_at']:
        [day, hour] = element.split()
        day_sin.append(sinc[day][0])
        day_cos.append(sinc[day][1])
        
        hour_sin.append(hours[hour][0])
        hour_cos.append(hours[hour][1])
    new_df = pd.DataFrame({'day_sin': day_sin, 'day_cos': day_cos, 'hour_sin': hour_sin, 'hour_cos': hour_cos})
    data = data.join(new_df)
    data = data.drop('created_at', axis=1)
    return data

# Text Process
<p dir='rtl'>
    به طور مشخص دو فیلد {عنوان، توضیحات} که تنها فیلدهایی هستند که مدل گوشی و وضعیت آن را مشخص می‌کنند قطعا نقش بسازایی در عملکرد شبکه دارند اما این فیلدها به این صورت خام قابل استفاده و ورودی دادن به شبکه نیستند لذا باید با روش مناسب اطلاعات مفیدی از آنها استخراج کنیم.
    <br><br>
    برای استخراج اطلاعات از این دو فیلد، ابتدا تمامی کلمات موجود در هر فیلد را جداگانه می‌خوانیم و تمامی کلمات به همراه تعداد تکرار شان را محاسبه می‌کنیم، سپس به دلیل تعداد زیاد کلمات و همچنین تعداد زیاد کلماتی که تکرار آنها بسیار پایین است، کلمه‌های کم‌تکرار را از لیست کلمات حذف می‌کنیم تا به این وسیله هم تعداد ویژگی ها کم شود و هم با حذف کلمات کم‌تکرار از overfitting
    مدل جلوگیری کنیم (زیرا اگر مثلا کلمه‌ای که یک بار تنها تکرار شده را به عنوان ویژگی به شبکه بدهیم شبکه می‌تواند آن داده را عملا حفظ کند و هر وقت آن ویژگی را دید قیمت آن محصول را خروجی دهد و این کار ارزش یادگیری ندارد).
    <br><br>
    کار با متن: به دلیل این که با متنی مواجه هستیم که بسیار نامتوازن و حاوی فاصله‌های نامناسب است قبل از هر بار خواندن هر متن ابتدا آن را به وسیله hazm.Normalizer()
    نرمالایز می‌کنیم تا کمی از نامتوازنی متن کاسته شود و سپس با استفاده از hazm.tokenize()
    آرایه حاوی کلمات متن را دریافت می‌کنیم و سپس روی هر کلمه hazm.Stemmer()
    اعمال می‌کنیم تا ریشه کلمه بدست آید و این ریشه کلمه معیار ما برای وقوع آن کلمه(ویژگی) در متن خواهد بود.
    <br><br>
    توضیح: علت معیار قرار دادن کلمات به عنوان ویژگی آن است که به طور کلی وقوع بعضی کلمات خاص می‌توانند معنای مثبت یا منفی در متن ایجاد کنند و البته که این روش خطا بسیار دارد ولی با توجه به امکانات ما در تحلیل متن و نوع متن نامتوازنی که در داده‌ها بود تصمیم گرفتم تا صرفا بر اساس وقوع کلمات ویژگی‌هایی را استخراج کنم.
    <br><br>
    در این جا کلماتی که وقوع آنها کمتر بوده از لیست کلی کلمات حذف شدند(برای عنوان، کلمات با تکرار بالای ۱۰۰ و برای توضیحات، کلمات با تکرار بالای ۵۰۰ استفاده شدند) و همچنن کلماتی که تنها یک کاراکتر داشتند نیز به طور کلی حذف شدند همچنین قطعا حذف کلمات پرتکراری که stop word
    هستند و عملا هرجایی می‌توانند بیایند و معنای خاصی به مدل نمی‌دهند می‌توانست بسیار کمک کند تا مدل سبک‌تری ساخته شود اما با توجه به این که کتابخانه این ابزار را نداشت نتوانستم این کار را انجام دهم و البته حضور این کلمات تداخلی در امر یادگیری شبکه ایجاد نمی‌کند زیر شبکه می‌تواند کارکرد آنها یا عدم کارکرد آنها را درک کند.
    <br><br>
    حال پس از نهایی کردن این لیست های کلمات که یکی بر اساس متن title
    و دیگری بر اساس متن desc
    هستند، تقریبا در مجموع به ۴۹۰ کلمه رسیدیم که در این مرحله به هر داده یک بردار oneHot
    به طول تعداد این کلمات اختصاص می‌دهیم تا وقوع این کلمات در هر یک از دو فیلد خود را ذخیره کند و این کار با استفاده از پیمایش روی داده ها انجام می‌شود.
    <br><br>
    به طور کلی علت این که متن های موجود در دو فیلد را جداگانه تحلیل و بررسی کردم این بود که به نظرم آمد دامنه کلمات این دو فیلد می‌تواند متفاوت باشد و وقوع کلمه در هر یک ممکن است بتواند معنای متفاوتی ایجاد کند.
</p>

In [79]:
def generate_library(data):
    normalizer = hazm.Normalizer()
    stemmer = hazm.Stemmer()
    
    title_wl = {}
    for sentence in data['title']:
        normal_sentence = normalizer.normalize(sentence)
        words = hazm.word_tokenize(normal_sentence)
        for word in words:
            sw = stemmer.stem(word)
            if sw not in title_wl:
                title_wl[sw] = 0
            title_wl[sw] += 1
    title_wl = {key:val for key, val in title_wl.items() if val > 100 and len(key) > 1}
    title_wl = {k: v for k, v in sorted(title_wl.items(), key=lambda item: item[1])}
    
    desc_wl = {}
    for sentence in data['desc']:
        normal_sentence = normalizer.normalize(sentence)
        words = hazm.word_tokenize(normal_sentence)
        for word in words:
            sw = stemmer.stem(word)
            if sw not in desc_wl:
                desc_wl[sw] = 0
            desc_wl[sw] += 1
    desc_wl = {key:val for key, val in desc_wl.items() if val > 500 and len(key) > 1}
    desc_wl = {k: v for k, v in sorted(desc_wl.items(), key=lambda item: item[1])}
                
    return title_wl.keys(), desc_wl.keys()

def encode_text(data):
    normalizer = hazm.Normalizer()
    stemmer = hazm.Stemmer()
    
    title_words, desc_words = generate_library(data)
    title_data = {}
    desc_data = {}
    for word in title_words:
        title_data['title_'+word] = [0 for _ in data['title']]
    for (i,sentence) in enumerate(data['title']):
        normal_sentence = normalizer.normalize(sentence)
        words = hazm.word_tokenize(normal_sentence)
        for word in words:
            sw = stemmer.stem(word)
            if 'title_'+sw in title_data:
                title_data['title_'+sw][i] = 1
    
    for word in desc_words:
        desc_data['desc_'+word] = [0 for _ in data['desc']]
    for (i, sentence) in enumerate(data['desc']):
        normal_sentence = normalizer.normalize(sentence)
        words = hazm.word_tokenize(normal_sentence)
        for word in words:
            sw = stemmer.stem(word)
            if 'desc_'+sw in desc_data:
                desc_data['desc_'+sw][i] = 1

    
    title_data = pd.DataFrame(title_data)
    desc_data = pd.DataFrame(desc_data)
    data = data.join(title_data)
    data = data.join(desc_data)
    data = data.drop(['title', 'desc'] , axis=1).dropna()
    return data

# Data Normalization
<p dir='rtl'>
    به طور کلی برای آموزش هر مدلی که بر پایه شبکه عصبی است باید داده‌ها را نرمالایز کنیم، که در این فرایند داده‌های هر فیلد به مقادیر بین {-۱و ۱} اسکیل می‌شوند به طوری که میانگین شان ۰ و واریانس شان ۱ شود و این عمل باعث می‌شود مدل روی این داده‌ها خوش فرم‌تر باشد و لذا بسیار زودتر و بهینه‌تر مدل همگرا شود، همچنین به  تحربه دیده شده که در بسیاری از مدل‌های شبکه عصبی که داده‌های یادگیری شان نرمال نشده شبکه هیچ چیزی یاد نگرفته و هیچ بهبودی در عملکردش نداشته است.
    <br><br>
    ما این عملیات نرمال‌سازی را پس از اتمام تمامی مراحل پیش‌پردازش روی داده‌ها، روی تمامی داده‌ها و تمامی فیلد‌های آنها انجام دادیم تا از این طریق اطمینان یابیم که روی تمامی داده‌هایمان دقیقا یک عملیات اسکیل انجام گرفته است. همچنین این تابع که نرمالایز کردن داده‌ها را انجام می‌دهد مدل scaler
    را نیز خروجی می‌دهد تا بتوانیم برای ورودی دادن داده‌هایی که قبلا نرمال نشده‌اند به شبکه ترین شده، ابتدا آنها را با گذراندن از این مدل نرمال کنیم.
</p>

In [80]:
def normalize_data(data):
    columns = data.columns
    scaler = StandardScaler()
    scaler.fit(data)
    x = scaler.transform(data)
    x = pd.DataFrame(x, columns=columns)
    return x, scaler

# Read Data & Perfrom PreProcess
<p dir='rtl'>
    در این مرحله داده‌ها را از فایل می‌خوانیم و توابعی که بالاتر تعریف کرده بودیم را روی آنها صدا می‌زنیم تا عملیات آماده‌سازی داده‌ها انجام گیرد. همچنین در این مرحله داده‌هایی که قیمت شان زیر ۱۰۰۰۰ را حذف می‌کنیم زیرا قیمت به این پایینی منطقی نیست و احتمالا یا داده قیمت ندارد و مقدارش -۱ است یا قیمت اشتباه وارد شده است.
    <br><br>
    علاوه بر این موارد، داده‌هایی که قیمت‌شان -۱ است را نیز جدا کردیم و در دیتافریم data_non_price 
    ذخیره کردیم.
</p>

In [110]:
data = pd.read_csv('mobile_phone_dataset.csv').drop(['Unnamed: 0'], axis=1).dropna().reset_index()
data = one_hot_encode_brand(data)
data = one_hot_encode_city(data)
data = created_at_encode(data)
data = encode_text(data)
data_non_price = data[data['price'] == -1].reset_index()
data = data[data['price'] > 10000].reset_index().dropna()

# Perform Data Normalization
<p dir='rtl'>
در این قسمت ستون قیمت از داده‌را به عنوان هدف(y
)
تعیین می‌کنیم وبرای ویژگی‌های داده‌ها نیز ستون قیمت را حذف کرده و بقیه را به عنوان
X
تعیین می‌کنیم.
</p>

In [111]:
data, data_scaler = normalize_data(data)
y = data['price']
X = data.drop('price', axis=1)

# Train/Test Set
<p dir='rtl'>
مطابق همیشه باید داده‌ها به دو بخش یادگیری و آزمایش تقسیم شوند تا بخشی از داده ها موجود باشند که مدل طی فرایند یادگیری آن‌را نمی‌بیند و از این داده‌های جدید برای مدل به منظور امتحان کردن و تست کارکرد مدل و محاسبه خطا استفاده می‌شود.
    <br><br>
    در اینجا ما ۲۰ درصد از داده‌ها را به عنوان داده آزمایش جدا می‌کنیم و در زمان بررسی عملکرد مدل از آن داده‌ها استفاده می‌کنیم. که اینجا تابع train_test_split()
    از کتابخانه sklearn
    این کار را برای ما انجام می‌دهد.
</p>

In [112]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Model
<p dir='rtl'>
    در اینجا ما مدل MLPRegressor()
    را به عنوان مدلی که بتواند این داده‌ها را تعداد شان تقریبا ۴۰۰۰۰ تا و تعداد ویژگی‌هایشان ۵۰۰ تا است را یاد بگیرد انتخاب کردیم.
    <br><br>
    این مدل در اصل یک شبکه چندلایه MLP
    فولی‌کانکتد است که این شبکه‌ها توانایی زیادی در تشخیص الگوهای موجود در داده‌ها دارند زیرا در چند لایه ما به متغیر های با درجات مختلف بر حسب تمامی وروردی‌ها می‌رسیم و این توانایی زیادی به شبکه می‌دهد.
    <br><br>
    ابعاد مدل: ما برای لایه‌های پنهان این مدل ابعاد ۱۰۰۰و۲۰۰۰و۱۰ را انتخاب کردیم تا در ابتدا به ویژگی‌های کلی از تمامی ویژگی‌ها برسد و سپس در لایه‌های بعدی به ویژگی قیمت که هدف ماست برسد. برای تابع غیرخطی بین لایه‌ها نیز شبکه به صورت پیش‌فرض از relu
    استفاده می‌کند که از مزایای این تابع می‌توان به موارد زیر اشاره کرد.
    <br><br>
    ۱-
    با توجه به این که مقدار تابع به ازای ورودی‌های منفی ۰ است، باعث ایجاد اسپارسیتی و ایجاد مقادیر بیشتری از صفر در شبکه و در نتیجه کاهش حجم محاسبات می‌شود. ۲- با توجه به اینکه در ناحیه مثبت شیب خط آن ثابت است و مانند sigmoid, tanh
   با افزایش مقدار ورودی شیب خط آن به صفر میل نمی‌کند و این باعث می‌شود شبکه vanishing gradient
    نکند و وزن‌های آن به صفر میل نکرده و سرعت یادگیری صفر نشود.
    <br><br>
    Adam Optimizer:
    مدل ما برای یادگیری از ابزار بهینه‌سازی adam
    استفاده می‌کند که روش کار آن بسیار شبیه به Stochastic Gradient Descent 
    است با این تفاوت های عمده که : ۱- برای هر وزن به طور جداگانه یک learning rate
    اختصاص داده و آن را استفاده می‌کند که این مورد برای یادگیری شبکه‌های با گرادیان‌های اسپارس بسیار مناسب عمل می‌کند و ما در اینجا با توجه به بردارهای ontHot
    که در ویژگی‌های داده‌ها بسیار استفاده کردیم شبکه و داده اسپارسی داریم و تعداد زیادی از ورودی‌های اولیه ما صفر هستند. ۲-از مومنت‌های مرتبه اول و دوم گرادیان‌ها برای تعیین این learning_rate 
    برای آپدیت وزن‌ها استفاده می‌کند.
    <br><br>
    در نهایت داده‌های ویژگی‌ها و هدف را به مدل می‌دهیم تا مدل روی آنها fit
    را انجام دهد و مدل ترین شود.
    <br><br>
    این مدل به طور کلی به صورت پیش‌فرض عملیات یادگیری را ادامه می‌دهد تا زمانی که برای ۱۰ بار متوالی مقدار هزینه روی داده‌ها تغییر نکند و همچنین در حالت ماکزیمم این تعداد مراحل تکرار یادگیری می‌تواند به ۲۰۰ برسد. ما پس از یادگیری مدل با این شرایط و مقایسه مقدار هزینه روی داده‌های یادگیری و آموزش به اختلاف بسیار زیاد بین این مقادیر پی بردیم لذا ما تعداد ماکزیمم تکرار ها را به ۱۰ کاهش دادیم که از این طیق میزان آن اختلاف کاهش یافت اما همچنان تفاوت معناداری موجود است که می‌تواند نشان‌دهنده overfitting
    باشد البته نباید از این نکته غافل شد که مدل پس از یادگیری همچنان دقت قایل قبولی روی داده‌های تست ارائه می‌دهد.
</p>

In [113]:
model = MLPRegressor(hidden_layer_sizes=(1000, 200, 10), max_iter=10)
model.fit(X_train, y_train)

/home/ho/ai/env/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPRegressor(hidden_layer_sizes=(1000, 200, 10), max_iter=10)

# Model Prediction
<p dir='rtl'>
    در اینحا تعداد تکرار های یادگیری مدل و هزینه مدل ترین شده روی داده‌های یادگیری و آزمایش را جداگانه بدست آوردیم و چاپ کردیم تا بتوان در مورد overfitting
    نظر داد.
</p>

In [114]:
print('num of iterations:', model.n_iter_)
y_pred = model.predict(X_train)
train_loss = MSE(y_train, y_pred)
print('train loss: ', train_loss)
y_pred = model.predict(X_test)
test_loss = MSE(y_test, y_pred)
print(' test loss: ', test_loss)

num of iterations: 10
train loss:  0.041991138293042636
 test loss:  0.29719289785080355


# Calculate RMSE, MAE
<p dir='rtl'>
    در اینجا یک تابع زدیم تا قیمت‌های پیش‌بینی شده مدل را برای برگرداند به قبل از اسکیل شدن و عملیات عکس نرمالیزیشن را برای ما انجام دهد. سپس قیمت های پیش‌بینی رو داده‌های آزمایش را با قیت‌های واقعی‌شان مقایسه کردیم.
    <br><br>
    ما معیارهای RMSE, MAE
    که به ترتیب جزر میانگین مربعات اختلاف داده‌ها و میانگین قدرمطلق اختلاف داده‌ها است را معیار این تفاوت قرار دادیم که به شرح زیر بدست آمدند.
</p>

In [118]:
def inverse_normalize_price(y):
    dummy = pd.DataFrame(np.zeros((len(y), data_scaler.n_features_in_)), columns=data.columns)
    dummy['price'] = y
    dummy = pd.DataFrame(data_scaler.inverse_transform(dummy), columns=data.columns)
    return dummy['price'].to_list()
y_test = y_test.to_list()
price_test = inverse_normalize_price(y_test)
price_pred = inverse_normalize_price(y_pred)

rmse = math.sqrt(MSE(price_test, price_pred))
print('rmse: ', rmse)
mae = MAE(price_test, price_pred)
print('mae: ', mae)

rmse:  299516.1926484881
mae:  199951.39069938275


# Predict Price for not having price data entries
<p dir='rtl'>
    در اینجا طبق خواسته سوال داده‌هایی که قیمت آنها -۱ بودند و تعیین شده بود را پیش‌بینی کردیم به این صورت که ابتدا آن داده‌ها را با نرمالایزر خود که روی داده‌ها فیت شده است نرمال کردیم و سپس قیمت های آن‌ها را با مدل پیش‌بینی کردیم و این قیمت‌ها را عکس نرمالایز کردی تا به قیمت‌های واقعی برسیم و آنها را در داده ‌های اولیه قرار دادیم و نتیجه را چاپ کردیم
</p>

In [131]:
non_normalized_data = data_scaler.transform(data_non_price)
non_normalized_data = pd.DataFrame(non_normalized_data, columns=data_non_price.columns)
non_prices = model.predict(non_normalized_data.drop('price', axis=1))
non_pred_prices = inverse_normalize_price(non_prices)
data_non_price['price'] = non_pred_prices
print(data_non_price)

      level_0  index  image_count         price  brand_Apple::اپل  \
0          23     23            1  4.700436e+05               0.0   
1          28     28            2  8.481692e+05               1.0   
2          32     32            0  9.895222e+05               1.0   
3          36     36            2  3.608371e+04               0.0   
4          44     44            0  4.363417e+05               0.0   
...       ...    ...          ...           ...               ...   
5883    59121  59121            3  2.698836e+05               0.0   
5884    59146  59146            3  1.494326e+05               0.0   
5885    59169  59169            4  1.617941e+06               1.0   
5886    59174  59174            0  1.433929e+06               0.0   
5887    59183  59183            0  4.461220e+05               0.0   

      brand_HTC::اچ‌تی‌سی  brand_Huawei::هوآوی  brand_LG::ال‌جی  \
0                     0.0                  0.0              0.0   
1                     0.0            

# PyTorch Implementation
<p dir='rtl'>
با توجه به این که با  ابزارهای موجود در اسکالرن قادر به تحلیل وضعیت overfitting
به خوبی نبودیم و با وجودی که به دقت قابل قبولی روی داده‌های تست رسیده‌ایم اما برای تحلیل قصد پیاده‌سازی مدل با استفاده از پایتورچ را داریم.
</p>

In [38]:
class MyDataset(Dataset):
    def __init__(self, data, train_subset=0.8):
        super().__init__()
        self.data = data
        self.columns = data.columns
        self.train_subset = train_subset
        self.X = data.drop('price', axis=1).to_numpy()
        self.y = data['price'].to_numpy()
    
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        return torch.tensor(self.X[idx]).float(), torch.tensor(self.y[idx]).float()

In [136]:
class Feedforward_net(nn.Module):
    def __init__(self, network_dims, activation=None):
        super().__init__()
        self.dims = network_dims
        self.activation = activation
        self.layers = nn.ModuleList([nn.Linear(self.dims[i], self.dims[i+1]) for i in range(len(self.dims)-2)])
        self.last_fc = nn.Linear(self.dims[-2], self.dims[-1])
        self.dropout = nn.Dropout(p=0.2)
        
    def forward(self, X):
        for layer in self.layers:
            X = layer(X)
            if self.activation == None:
                X = F.relu(X)
            elif self.activation == "leakyRelu":
                X = F.leaky_relu(X)
            elif self.activation == "tanh":
                X = torch.tanh(X)
#         X = self.dropout(X)
        X = self.last_fc(X)
        return X

In [137]:
batch_size = 64
dataset = MyDataset(data)
train_set, test_set = random_split(dataset, [len(data)-int(len(dataset)*0.2), int(len(dataset)*0.2)])
train_data_loader = DataLoader(train_set, batch_size=batch_size, sampler=SubsetRandomSampler(range(len(train_set))), drop_last=True)
test_data_loader = DataLoader(test_set, batch_size=batch_size, drop_last=False)

n_epochs = 10
learning_rate = 0.01
device = 'cuda' if torch.cuda.is_available() else 'cpu'
criterion = nn.MSELoss()

In [138]:
def train(model, optimizer, train_loader, test_loader, device=device, criterion=criterion, n_epoch=10):
    total_time = 0  
    train_losses = []
    test_losses = []
    for epoch in range(n_epoch):
        epoch_loss = 0
        d1 = datetime.now()
        model.train()
        for (batch, target) in tqdm(train_loader):
            batch = batch.to(device)
            target = target.to(device)
            predicts = model(batch)
            optimizer.zero_grad()
            loss = criterion(predicts, target)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()  
        
        average_loss = epoch_loss/len(train_loader)
        train_losses.append(average_loss)
        d2 = datetime.now()
        delta = d2 - d1
        seconds = float(delta.total_seconds())
        total_time += seconds
        print('epoch %d, train_loss: %.3f, time elapsed: %s seconds' % (epoch + 1, average_loss, seconds))
        
        model.eval()
        with torch.no_grad():
            epoch_loss = 0
            for (batch, target) in test_loader:
                batch = batch.to(device)
                target = target.to(device)
                predicts = model(batch)
                loss = criterion(predicts, target)
                epoch_loss += loss.item()
            average_loss = epoch_loss/len(test_loader)
            test_losses.append(average_loss)
            print('test_loss: %.3f'%(average_loss))
            
    print('total training time: %.3f minutes' % (total_time / 60))
    plt.plot(test_losses, label='test', color='r')
    plt.plot(train_losses, label='train', color='b')
    plt.show()
    return losses

In [139]:
dmodel = Feedforward_net([512, 1000, 200, 1]).to(device)
criterion = nn.MSELoss()
optimizer = Adam(dmodel.parameters(), lr=0.001)
losses = train(dmodel, optimizer, train_data_loader, test_data_loader)

RuntimeError: CUDA error: unspecified launch failure

In [67]:
y_pred = dmodel(torch.tensor(X_test.to_numpy()).float().to(device))
print(type(criterion))
print(type(y_pred))
print(type(y_test))
# loss = criterion(y_pred, y_test.to_numpy())
# print('loss = ', loss)
def inverse_normalize_price(y):
    dummy = pd.DataFrame(np.zeros((len(y), data_scaler.n_features_in_)), columns=data.columns)
    dummy['price'] = y
    dummy = pd.DataFrame(data_scaler.inverse_transform(dummy), columns=data.columns)
    return dummy['price'].to_list()
# y_test = y_test.to_list()
print(type(y_pred), type(y_test))
price_test = inverse_normalize_price(y_test)
price_pred = inverse_normalize_price(y_pred)

mean_error = math.sqrt(MSE(price_test, price_pred))
print('mean error = ', mean_error)
for i in range(len(price_test)):
    print(price_test[i], price_pred[i])

<class 'torch.nn.modules.loss.MSELoss'>
<class 'torch.Tensor'>
<class 'pandas.core.series.Series'>
<class 'torch.Tensor'> <class 'pandas.core.series.Series'>


TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

In [ ]:
torch.cuda.is_available()